In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common import keys
from time import sleep
import os
os.environ['PATH'] += ':.' 

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get('http://web.whatsapp.com')

In [ ]:
# signed_in = driver.find_elements_by_xpath("//*[contains(text(), 'Keep me')]")[0]
# signed_in.click()

In [ ]:
print("CONFIRME O QR CODE E DEPOIS CLIQUE NO CHAT DO GRUPO QUE QUER CRAWLEAR!")
sleep(1)

In [ ]:
body = driver.find_element_by_tag_name('body')
chat_area_father = body.find_element_by_class_name('copyable-area')
chat_area = chat_area_father.find_element_by_xpath('div[1]') # get 1st div child
chat_area.get_attribute('class')

In [ ]:
messages = driver.find_elements_by_class_name('message-in')
len(messages)


In [ ]:
def sleep_until_messages_loaded(body):
    for i in range(10):
        try: body.find_element_by_xpath('//div[contains(@title, "loading messages")]')
        except selenium.common.exceptions.NoSuchElementException: return True
        sleep(0.5)
    return False
els = [] # RENAME ELS
sleep_until_messages_loaded(body)

In [ ]:
### RENAME FUNCTION
chat_area.click() # TRY TO CLICK IN an area withou conflict
for i in range(150):
    removed_els = driver.execute_script('''
function _x(STR_XPATH) {
    var xresult = document.evaluate(STR_XPATH, document, null, XPathResult.ANY_TYPE, null);
    var xnodes = [];
    var xres;
    while (xres = xresult.iterateNext()) {
        xnodes.push(xres);
    }

    return xnodes;
}
return _.map(_x('html/body/div/div/div/div[3]/div/div[2]/div/div/div[3]/div').slice(10,100000)
        ,function(x){
            if(x) {a = x.outerHTML; x.remove(); return a}
        })
''')
    removed_els.reverse()
    els += removed_els
    [body.send_keys(keys.Keys.PAGE_UP) for _ in range(5)]
    sleep(0.5)
    sleep_until_messages_loaded(body)
    print(i, flush=True, end=',')
    if (i % 10) == 0: chat_area.click(), print()

print('\n Length');
print(len(els))

In [ ]:
import copy
import settings
FILENAME = settings.GROUP_NAME + '.html'

r_els = copy.copy(els)
r_els.reverse()
with open(FILENAME, 'w+') as f:
    f.writelines(r_els)
    print(f'{FILENAME} written!')

## Extras

In [ ]:
############## Extras ##############

import base64
def get_blob_content(driver, uri) -> bytes:
    """
    Use to grab files such as images from blob links
    get_blob_content("blob:https://web.whatsapp.com/cf8679c6-e3cb-4a30-91ed-3a67de1d5dd4")
    """
    result = driver.execute_async_script("""
            var uri = arguments[0];
            var callback = arguments[1];
            var toBase64 = function(buffer){for(var r,n=new Uint8Array(buffer),t=n.length,a=new Uint8Array(4*Math.ceil(t/3)),i=new Uint8Array(64),o=0,c=0;64>c;++c)i[c]="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/".charCodeAt(c);for(c=0;t-t%3>c;c+=3,o+=4)r=n[c]<<16|n[c+1]<<8|n[c+2],a[o]=i[r>>18],a[o+1]=i[r>>12&63],a[o+2]=i[r>>6&63],a[o+3]=i[63&r];return t%3===1?(r=n[t-1],a[o]=i[r>>2],a[o+1]=i[r<<4&63],a[o+2]=61,a[o+3]=61):t%3===2&&(r=(n[t-2]<<8)+n[t-1],a[o]=i[r>>10],a[o+1]=i[r>>4&63],a[o+2]=i[r<<2&63],a[o+3]=61),new TextDecoder("ascii").decode(a)};
            var xhr = new XMLHttpRequest();
            xhr.responseType = 'arraybuffer';
            xhr.onload = function(){ callback(toBase64(xhr.response)) };
            xhr.onerror = function(){ callback(xhr.status) };
            xhr.open('GET', uri);
            xhr.send();
            """, uri)
    if type(result) == int :
        raise Exception("Request failed with status %s" % result)
    return base64.b64decode(result)

def bind(instance, func, asname): 
    setattr(instance, asname, func.__get__(instance, instance.__class__))

def bind_driver(driver):
    bind(driver, get_blob_content, 'get_blob_content')
    
def display_image(bytes):
    from IPython.display import Image
    with open('/tmp/a.jpg', 'wb') as f: f.write(bytes)
    Image('/tmp/a.jpg')